In [8]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import glob

# Function to extract SIFT features from an image
def extract_sift_features(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors

# Function to extract SURF features from an image
def extract_surf_features(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    surf = cv2.xfeatures2d.SURF_create()
    keypoints, descriptors = surf.detectAndCompute(gray, None)
    return descriptors

# Get image paths
image_paths_train = glob.glob('./Assignment2_BikeHorses/Assignment2_BikeHorses/cifar-10-python/cifar-10-batches-py/data_batch_*')
image_paths_test = glob.glob('./Assignment2_BikeHorses/Assignment2_BikeHorses/cifar-10-python/cifar-10-batches-py/test_batch')

# Extract features from images
all_descriptors = []
for image_path in image_paths_train + image_paths_test:
    descriptors = extract_sift_features(image_path)
    if descriptors is not None:
        all_descriptors.extend(descriptors)

# Convert descriptors to numpy array
all_descriptors = np.array(all_descriptors)

# Perform K-means clustering
kmeans = KMeans(n_clusters=100)
kmeans.fit(all_descriptors)

# Function to generate bag of visual words representation
def bag_of_visual_words(image_paths, kmeans):
    features = []
    for image_path in image_paths:
        descriptors = extract_sift_features(image_path)
        if descriptors is not None:
            words = kmeans.predict(descriptors)
            histogram, _ = np.histogram(words, bins=range(101))
            features.append(histogram)
    return np.array(features)

# Generate features for training and testing data
X_train = bag_of_visual_words(image_paths_train, kmeans)
X_test = bag_of_visual_words(image_paths_test, kmeans)

# Dummy labels for demonstration
y_train = np.random.randint(0, 2, size=len(image_paths_train))
y_test = np.random.randint(0, 2, size=len(image_paths_test))

# Train SVM classifier
clf = make_pipeline(StandardScaler(), SVC())
clf.fit(X_train, y_train)

# Evaluate on test data
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
